In [15]:
import requests

# Daily Recommended Calorie Calculator

In [16]:
def calories_macros_recommended(user_age,user_gender,user_height,user_weight,user_activity_level,user_goal):

    #REE : the number of calories it takes to run your body without movement

    # user inputs needed to calculate REE
    #user_age,user_gender,user_height,user_weight
    # REE Recommended
    # 10 x weight (kg) + 6.25 x height (cm) – 5 x age (y) + 5 = REE

    if user_gender == "Men":
        user_ree = 10*user_weight + 6.25*user_height - 5*user_age + 5
    user_ree = 10*user_weight + 6.25*user_height - 5*user_age - 161 #user is a woman
    
    
    # TDEE : the number of calories your particular body burns in a day
    
    #activity levels : 
    # Sedentary
    # Just everyday activities like a bit of walking, a couple of flights of stairs, eating, talking, etc. (REE X 1.2)
    # Light activity
    # Any activity that burns an additional 200-400 calories for females or 250-500 calories for males. (REE x 1.375)
    # Moderate activity
    # Any activity that burns an additional 400-650 calories for females or 500-800 calories for males. (REE x 1.55)
    # Very Active
    # Any activity that burns an additional 650+ calories for females or 800+ calories for males. (REE x 1.725) 

    activity_level_factor = {
        'Sedentary': 1.2,
        'Light activity': 1.375,
        'Moderate activity': 1.55,
        'Very Active': 1.725   
    }

    #user TDEE
    user_tdee = user_ree * activity_level_factor[f'{user_activity_level}']
    
    #User's goal: lose, maintain or gain weight

    goals_factor = {
        "Weight loss": 0.8,
        "Maintain": 1,
        "Weight gain": 1.1
    }

    user_final_tdee = user_tdee * goals_factor[f'{user_goal}']
    #Calculate the recommended protein / fat / carbs intake based on the calories
    # 1 g protein = 4 Calories
    # 1 g Carbohydrate = 4 Calories
    # 1 g fat = 9 Calories

    # Proteins :  Most people can use a more balanced approach and .825 g protein per pound
    weight_lbs = user_weight * 2.2 
    daily_protein_intake = weight_lbs * 0.825
    daily_protein_cals = daily_protein_intake * 4

    # Fats : Most research (and again, there are a lot of opinions) supports that 20%-30% of overall TDEE calories 
    # should come from the fat macro. Let’s use 25% and a middle-of-the-road starting point. 
    daily_fat_cals = user_tdee * 0.25
    daily_fat_intake = daily_fat_cals / 9

    # Carbs : We allocate the rest of our calories (originally calculated from our TDEE) to carbohydrates.
    daily_carb_cals = user_tdee - daily_protein_cals - daily_fat_cals
    daily_carb_intake = daily_carb_cals / 4
    individual_ideal={}
    individual_ideal['user_tdee']=user_tdee
    individual_ideal['daily_protein_cals']=daily_protein_cals
    individual_ideal['daily_fat_cals']=daily_fat_cals
    individual_ideal['daily_carb_cals']=daily_carb_cals
    %store individual_ideal
    return individual_ideal

# Food Macro Calculator For Already Eaten Foods

In [17]:
def calculator(foods):
    cals = 0
    proteins = 0
    fats = 0 
    carbs = 0
    fibers = 0
    app_id="4e07f131"
    app_key="6f9bb3ba56ff2c9786adc5ade08cf1d2"
    for food in foods:
        
        url = f'https://api.edamam.com/api/food-database/v2/parser?app_id={app_id}&app_key={app_key}%09&ingr={food}&nutrition-type=cooking'
        r = requests.get(url).json()
        
        cal = r['hints'][0]['food']['nutrients']['ENERC_KCAL']
        protein = r['hints'][0]['food']['nutrients']['PROCNT']
        fat = r['hints'][0]['food']['nutrients']['FAT']
        carb = r['hints'][0]['food']['nutrients']['CHOCDF']
        fiber = r['hints'][0]['food']['nutrients'].get('FIBTG',0)
        cals += cal
        proteins += protein
        fats += fat 
        carbs += carb
        fibers += fiber
        individual_junk={}
        individual_junk['cal']=cal
        individual_junk['protein']=protein
        individual_junk['carb']=carb
        individual_junk['fats']=fats
        %store individual_junk
        return individual_junk

## Remaining cals and macros

In [24]:
remainder={}
remainder['rem_cals']=individual_ideal['user_tdee']-individual_junk['cal']
remainder['rem_prot']=individual_ideal['daily_protein_cals']-individual_junk['protein']
remainder['rem_fats']=individual_ideal['daily_fat_cals']-individual_junk['fats']
remainder['rem_carbs']=individual_ideal['daily_carb_cals']-individual_junk['carb']
%store remainder

Stored 'remainder' (dict)


# Food Recipe API

## inputs: food available, calories remaining

In [28]:
def food_recipe(food_choice,predicted_calories):  #predicted_calories is recieved from the ml model

    recipe_key = '1c4f674c'
    recipe_id = 'f4063bcf2c650488f044cb2c2747fea3'
    calories = "400-500"
#     food_choice = "pu"

    url_2 = f"https://api.edamam.com/api/recipes/v2?type=public&q={food_choice}&app_id={recipe_key}&app_key={recipe_id}&calories={calories}"
    r_2 = requests.get(url_2).json()


    recipe_titles = r_2['hits']
    recipe=[]
    for dicts in recipe_titles:
        recipe.append(dicts['recipe']['label'])
    return recipe

In [27]:
food_recipe('pumpkin,chicken','500-600')

['Thai-style Pumpkin, Chicken And Coconut Soup recipes',
 'Dinner Tonight: Meatless Pozole Verde (Hominy Soup)',
 'Roast Pumpkin with Cheese "Fondue"',
 'Chicken With Pumpkin & Chickpeas',
 'Pumpkin and Mushroom Risotto',
 'Olla Gitana (Gypsy Pot)',
 'Pumpkin Sweet Potato Mash',
 'Sausage, pumpkin and sage casserole',
 'Red Cabbage Salad with Spicy Grilled Chicken and Pepitas recipes',
 'Pumpkin-Chickpea Curry',
 'Cook the Book: Coconut Rice with Variations',
 'Hearts of palm & chicken chopped salad',
 'Mediterranean Chicken Stew with Prunes and Pumpkin',
 'Pumpkin Soup with Sage and Ham recipes',
 'Roasted Squash and Pepita Pesto Dip recipes',
 'Pumpkin Risotto with Sage and Cherry Tomatoes',
 'Open Faced Smoked Chicken and Mustard Fruits Sandwich',
 'Quinoa salad with avocado mayo',
 'Creamy Pumpkin Spaghetti with Garlic Kale recipes',
 'Chicken with Squash-Barley Pilaf']